In [2]:
import pandas as pd
import numpy as np
import plotly.express as px
import seaborn as sns
import matplotlib.pyplot as plt
import pandasdmx as sdmx
from functools import reduce

/opt/homebrew/Caskroom/miniconda/base/envs/wafflers/lib/python3.11/site-packages/pandasdmx/remote.py:11: RuntimeWarning: optional dependency requests_cache is not installed; cache options to Session() have no effect
  warn(


In [7]:
def melt_smdx_dataframe(df: pd.DataFrame) -> pd.DataFrame:
	"""
	Given an ESTAT smdx dataframe, convert the datetimes to years and melt

	:param df: The raw SDMX parsed dataframe from ESTAT
	:returns: A melted dataframe with the columns of:
		`year` - the year of the observation
		`geo` - the country of the observation
		`value` - the value of the observation
	"""
	df = df.reset_index()
	df["year"] = df["TIME_PERIOD"].dt.year
	df = df.drop("TIME_PERIOD", axis=1)
	return pd.melt(df, id_vars="year")

def merge_dataframes(dataframes: list[pd.DataFrame]) -> pd.DataFrame:
	"""
	"""
	for i, df in enumerate(dataframes):
		df.columns = ["geo", "year", i]

	merged_df = reduce(lambda l, r: pd.merge(l, r, left_on=["year", "geo"], right_on=["year", "geo"]), dataframes)
	return merged_df

def fill_holes(df: pd.DataFrame) -> pd.DataFrame:
	"""
	"""
	lin_reg = lambda X, Y: np.matmul(np.linalg.inv(np.matmul(X.T, X)), np.matmul(X.T, Y))

	dfs = []

	for name, group in df.groupby('geo'):
		cols = [[name for _ in range(len(group.index))]]
		for i in range(1, len(group.columns)):
			d = group.iloc[:, i:i+1].to_numpy()

			missing_mask = np.isnan(d) | (d == 0)
			present_mask = ~missing_mask

			missing_mask = missing_mask.reshape(1, -1)[0]
			present_mask = present_mask.reshape(1, -1)[0]

			if not np.any(missing_mask):
				d = d.reshape(1, -1)[0]
				cols.append(d)
				continue

			if not np.any(present_mask):
				d = d.reshape(1, -1)[0]
				cols.append(d)
				continue

			x_present = np.pad(np.arange(len(d))[present_mask].reshape(-1, 1), ((0, 0), (1, 0)), mode="constant", constant_values=1)
			y_present = d[present_mask]

			w = lin_reg(x_present, y_present)

			x_missing = np.pad(np.arange(len(d))[missing_mask].reshape(-1, 1), ((0, 0), (1, 0)), mode="constant", constant_values=1)
			y_missing_pred = np.matmul(x_missing, w)

			d[missing_mask] = y_missing_pred
			d = d.reshape(1, -1)[0]

			cols.append(d)
			
		dfs.append(pd.DataFrame(cols).T)	
		
	df_unswissed = pd.concat(dfs, axis=0)
	df_unswissed.columns = df.columns
	return df_unswissed

def standardize(df: pd.DataFrame) -> pd.DataFrame:
	"""
	"""
	df_standard = pd.DataFrame()
	for feat in df.columns:
		if feat == "geo": continue
		df_standard[f'{feat}'] = ((df[feat] - df[feat].mean()) / df[feat].std())
	df_standard["geo"] = df["geo"]

	return df_standard

In [27]:
estat = sdmx.Request("ESTAT")
resp = estat.data(
	"ENV_AIR_GGE",
	key={
		"unit": "THS_T",
		"freq": "A",
		"src_crf": "TOTX4_MEMONIA",
		"airpol": "GHG"
	}
)
emission_df = resp.to_pandas(datetime={'dim': 'TIME_PERIOD'}).droplevel(level=['unit', 'freq', 'src_crf', 'airpol'], axis=1)
melted_emissions_df = melt_smdx_dataframe(emission_df)

resp = estat.data(
		"NRG_D_HHQ",
		key={
			"siec": "TOTAL",
			"unit": "TJ",
			"nrg_bal": "FC_OTH_HH_E",
			"freq": "A",
		}
	)
household_energy_df = resp.to_pandas(datetime={'dim': 'TIME_PERIOD', 'freq': 'freq'}).droplevel(level=["siec", "unit", "nrg_bal"], axis=1)
melted_household_energy_df = melt_smdx_dataframe(household_energy_df)

resp = estat.data(
	"TEN00127",
	key={
		"unit": "KTOE",
		"freq": "A",
		"siec": "O4652XR5210B",
		"nrg_bal": "FC_TRA_ROAD_E"
	}
)
gas_df = resp.to_pandas(datetime={'dim': 'TIME_PERIOD'}).droplevel(level=['unit', 'freq', 'siec', "nrg_bal"], axis=1)
melted_gas_df = melt_smdx_dataframe(gas_df)

merged_df = merge_dataframes([melted_emissions_df, melted_household_energy_df, melted_gas_df])
merged_df.columns = ["year", "geo", "emissions", "energy", "gas"]
merged_df = merged_df.drop(merged_df[(merged_df.geo == "EU27_2020") | (merged_df.geo == "EU20")].index)
merged_df = merged_df.drop("year", axis=1)
merged_df = fill_holes(merged_df)
standard_df = standardize(merged_df)

df_dummies = pd.get_dummies(standard_df, dtype=int, columns=["geo"])
df_dummies = df_dummies.fillna(0)

#X = np.pad(df_dummies.iloc[:, 1:].to_numpy(dtype=np.float64), ((0,0), (1,0)), mode="constant", constant_values=1)
X = np.pad(standard_df.iloc[:,1:3].to_numpy(dtype=np.float64), ((0,0), (1,0)), mode="constant", constant_values=1)
y = np.array(df_dummies["emissions"], dtype=np.float64)

m = np.matmul(np.linalg.inv(np.matmul(X.T, X)), np.matmul(X.T, y))
m

2024-06-05 23:46:41,043 pandasdmx.reader.sdmxml - INFO: Use supplied dsd=… argument for non–structure-specific message
2024-06-05 23:46:41,240 pandasdmx.reader.sdmxml - INFO: Use supplied dsd=… argument for non–structure-specific message
/opt/homebrew/Caskroom/miniconda/base/envs/wafflers/lib/python3.11/functools.py:909: FutureWarning: 'A' is deprecated and will be removed in a future version, please use 'Y' instead.
  return dispatch(args[0].__class__)(*args, **kw)
2024-06-05 23:46:41,456 pandasdmx.reader.sdmxml - INFO: Use supplied dsd=… argument for non–structure-specific message
/var/folders/jk/vqmr1l1d1dnflmyh1qdjmhk40000gn/T/ipykernel_97540/3566454452.py:46: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_dummies = df_dummies.fillna(0)


array([3.89988871e-17, 4.61239971e-01, 5.29738712e-01])

In [19]:
from sklearn.metrics import r2_score

In [31]:
np_remove = lambda a, i: np.concatenate([a[:i,], a[i + 1:,]])
lin_reg = lambda X, Y: np.matmul(np.linalg.inv(np.matmul(X.T, X)), np.matmul(X.T, Y))

def loo_cv_pred(X, Y):
	"""
	Predict Y values using leave one out cross validation

	:param X: The X features array (including bias column)
	:param Y: The true Y values
	:return: An array of the predicted Y-Vals
	"""
	y_pred = []
	for i in range(len(X)):
		holdout_X = X[i]
		
		loo_X = np_remove(X, i)
		loo_y = np_remove(Y, i)
		loo_b = lin_reg(loo_X, loo_y)

		y_hat = np.matmul(holdout_X, loo_b)
		y_pred.append(y_hat)
	
	return y_pred


In [32]:
preds = loo_cv_pred(X, y)
r2_score(y, preds)

0.9607943812769059

In [33]:
(y - preds).mean()

-0.0002890124341005411